In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://delta-hotels.marriott.com/our-locations/"
driver.get(url)
time.sleep(5)
viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[5]/pacsys-directory/div[3]/button')[0]
viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
achotels_df = pd.DataFrame(hotel_list)
achotels_df

,name,url
0,Delta Hotels Kunming,https://www.marriott.com/KMGDE
1,Delta Hotels Shanghai Baoshan,https://www.marriott.com/SHABS
2,Delta Hotels by Marriott Frankfurt Offenbach,https://www.marriott.com/FRADE
3,Delta Hotels Istanbul Halic,https://www.marriott.com/ISTDK
4,Delta Hotels Cheltenham Chase,https://www.marriott.com/GLODE
...,...,...
84,Delta Hotels Huntington Downtown,https://www.marriott.com/HTSDH
85,Delta Hotels Green Bay,https://www.marriott.com/GRBDE
86,Delta Hotels Milwaukee Northwest,https://www.marriott.com/MKEDM
87,Milwaukee Airport Hotel,https://www.marriott.com/MKEDA


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in achotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Delta Hotels'
        except:
            hotel['brand'] = 'Delta Hotels'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df      

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Kunming', 'url': 'https://www.marriott.com/KMGDE', 'street': 'Longhui Road 206', 'locality': 'Kunming', 'state': '', 'postalcode': '650000', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Delta Hotels by Marriott Kunming is located in the commercial hub of Kunming Economic &amp; Technology Development Zone. It takes 40 minutes to drive to Kunming Changshui International Airport and Kunming South Railway Station, as well as a 30-minute drive to Kunming Dianchi Lake International Convention &amp; Exhibition Centre. Our hotel features 639 guestrooms, covering 7 room types. While seated at the Tea Library, guests will enjoy delicious drinks with access to a large range of books to read. We are unique in that our hotel is connected and shares facilities with Sheraton Kunming. The fusion of these two Marriott brands allows guests to take advantage of extra amenities such as the Sheraton

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Nottingham Belfry', 'url': 'https://www.marriott.com/EMADE', 'street': 'Mellors Way off Woodhouse Way', 'locality': 'Nottingham', 'state': '', 'postalcode': 'NG8 6PY', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Delta by Marriott Hotel Nottingham Belfry is the ideal choice for business and leisure.  Guests can expect a unique blend of modernity, luxury and efficiency, as well as a team of professional staff, intuitive of your needs throughout your stay.  As one of the largest conference venues in Nottinghamshire, Delta Hotel boasts 10 flexible meeting rooms suitable for an array of events with numbers from 6 to 700 and all rooms are air conditioned and equipped with complimentary Wi-Fi.  The menu variety provides excellent choice for all of your delegates, ensuring we compliment your occasion perfectly.  Our dining options in the Lawrence Restaurant or Oaks Bar provide the perf

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Calgary Downtown', 'url': 'https://www.marriott.com/YYCBV', 'street': '209 Fourth Avenue SE', 'locality': 'Calgary', 'state': 'Alberta', 'postalcode': 'T2G 0C6', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/yycbv-delta-hotels-calgary-downtown/?directPageRequest=true', 'coordinate3': '51.048675', 'coordinate4': '-114.060119', 'description': "[ Travel on your terms at Delta Hotels Calgary Downtown. Newly redesigned, thoughtfully appointed and impeccably located, ours is a hotel experience that simply works. Connected to the +15 Skyway, we make it easy for you to get to know the downtown Calgary area. Our hotel restaurant, The Shoe &amp; Canoe Public House, celebrates the best of Western Canada, with approachable cuisine and our exclusive Stargazer Ale. Keep up with your fitness goals with our dedicated fitness floor - includes indoor pool, fully equipped gym with natural light and a yoga/stretching r

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Edmonton South Conference Centre', 'url': 'https://www.marriott.com/YEGES', 'street': '4404 Gateway Boulevard NW', 'locality': 'Edmonton', 'state': 'Alberta', 'postalcode': 'T6H 5C2', 'coordinate1': '-113.492662', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Delta Hotels Edmonton South Conference Centre welcomes you with well-considered amenities and award-winning service. Situated a short drive from the airport and close to attractions including West Edmonton Mall and the University of Alberta, our hotel is well suited to business trips and family vacations. Stay warm with our heated underground, complimentary parking. Settle in to pet-friendly rooms and suites that work, with free Wi-Fi, pillowtop beds, flat-screen TVs and more. Take advantage of our 24-hour fitness center and a heated indoor pool. Our hotel is the city's premier destination for meetings and events, with over 25,000 square feet of 

{'brand': 'Delta Hotels', 'name': 'Delta Hotels by Marriott Grand Okanagan Resort', 'url': 'https://www.marriott.com/YLWOK', 'street': '1310 Water Street', 'locality': 'Kelowna', 'state': 'British Columbia', 'postalcode': 'V1Y 9P3', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ylwok-delta-hotels-grand-okanagan-resort/?directPageRequest=true', 'coordinate3': '49.891541', 'coordinate4': '-119.496875', 'description': "[ Immerse yourself in lakeside ambiance in the midst of British Columbia's Wine Country at Delta Hotels by Marriott Grand Okanagan Resort. Set on Okanagan Lake and surrounded by picturesque mountains in downtown Kelowna, our resort offers stylish hotel rooms and suites with deluxe amenities - many with balconies. Feast at our popular hotel restaurant and sip cocktails or wine on our lakefront patio. Stay fit in our 24/7 fitness centre. Take a dip in our sparkling indoor and outdoor pools. Indulge in a massage at our on-site, full-serv

...

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Utica', 'url': 'https://www.marriott.com/UCADE', 'street': '200 Genesee Street', 'locality': 'Utica', 'state': 'New York', 'postalcode': '13502', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ucade-delta-hotels-utica/?directPageRequest=true', 'coordinate3': '43.100875', 'coordinate4': '-75.232497', 'description': '[ Simple Made Perfect at the Delta Marriott Utica! Our goal is to make your experience streamlined and flawless every time you stay with us. Step into our newly renovated full service Hotel located in downtown Utica, we invite you to enjoy stunning surrounding both inside and out. Visit a variety of nearby attractions, such as the Stanley Theater, the Adirondack Bank Center - home to the Utica Comets and many local restaurants, all within walking distance of our hotel!We have created a seamless full-service hotel experience that has been thoughtfully designed with the essential needs of th

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Dallas Southlake', 'url': 'https://www.marriott.com/DFWDL', 'street': '251 East State Hwy 114', 'locality': 'Southlake', 'state': 'Texas', 'postalcode': '76092', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Inspired by a journey through thick, dense forest, the Delta Hotel by Marriott Dallas Southlake will surprise and delight with hidden natural gems around every corner.  Thoughtful design and intricate details appoint 240 guest rooms and public spaces. Get a sense of the surrounding area and the rich history Southlake and Dallas- Fort Worth have to offer while visiting charming local towns and experiencing the great outdoors. With over 17,000 sq ft of flexible meeting space, our convenient coffee shop, on-site restaurant and outdoor pool and lounge area provides seamless functionality.At the Delta Hotel by Marriott Dallas Southlake, we create a seamless full-service hotel expe

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Virginia Beach Bayfront Suites', 'url': 'https://www.marriott.com/ORFDB', 'street': '2800 Shore Drive', 'locality': 'Virginia Beach', 'state': 'Virginia', 'postalcode': '23451', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ With panoramic water views, Delta Hotels by Marriott Virginia Beach Bayfront Suites is an oasis on the shores of the breathtaking Chesapeake Bay. Embrace effortless relaxation in beautifully appointed suites with complimentary Wi-Fi, signature amenities, luxurious bedding and breathtaking bay views. Thrill your palate with fresh oysters, fish and coastal cuisine at our distinctive hotel restaurant, featuring inspiring water views. Enjoy an ideal location near diverse water activities on Chesapeake Bay, near First Landing State Park and Virginia Beach Sportsplex. Host flawless weddings, meetings and social galas in our dynamic indoor event space or inspiring ou

{'brand': 'Delta Hotels', 'name': 'Delta Hotels Milwaukee Northwest', 'url': 'https://www.marriott.com/MKEDM', 'street': 'N88 W14750 Main Street', 'locality': 'Menomonee Falls', 'state': 'Wisconsin', 'postalcode': '53051', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/mkedm-delta-hotels-milwaukee-northwest/?directPageRequest=true', 'coordinate3': '43.177772', 'coordinate4': '-88.093336', 'description': "[ A seamless experience awaits you at Delta Hotels Milwaukee Northwest.  Situated in Menomonee Falls, our hotel offers thoughtful design with the essential needs of frequent traveler in mind. Settle into our modern yet convenient designed rooms and suites with smart amenities including fast, free Wi-Fi, complimentary bottled water, over-sized TVs and plush Marriott brand bedding. Waterlin Coffee Bar &amp; Bistro, our signature restaurant, showcases classic WI comfort dishes with modern twists—locally sourced and inspired. The cocktail menu offers 

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Delta Hotels,Delta Hotels Kunming,https://www.marriott.com/KMGDE,Longhui Road 206,Kunming,,650000,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Kunming is located ...,N/A,N/A,N/A
1,Delta Hotels,Delta Hotels Shanghai Baoshan,https://www.marriott.com/SHABS,"No 1 Lane 888 Haijiang Road, Baoshan District",Shanghai,,200940,Reviews,https://www.marriott.com/hotels/maps/travel/sh...,31.3954603,121.4884866,"[ The hotel started trial opening on Feb 1,201...",N/A,4.7,4.5
2,Delta Hotels,Delta Hotels by Marriott Frankfurt Offenbach,https://www.marriott.com/FRADE,Kaiserleistrasse 45,Offenbach am Main,,63067,Reviews,https://www.marriott.com/hotels/maps/travel/fr...,50.108019,8.73165,[ Make your time in Germany memorable with a s...,N/A,4.7,4.6
3,Delta Hotels,Delta Hotels Istanbul Halic,https://www.marriott.com/ISTDK,Merkez Mahallesi Ayazma Caddesi Cuha Sokak No....,Istanbul,,34406,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Istanbul Halic is l...,N/A,N/A,N/A
4,Delta Hotels,Delta Hotels Cheltenham Chase,https://www.marriott.com/GLODE,Shurdington Road,Brockworth,,GL3 4PB,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Cheltenham Chase pr...,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Delta Hotels,Delta Hotels Huntington Downtown,https://www.marriott.com/HTSDH,800 Third Avenue,Huntington,West Virginia,25701,Reviews,https://www.marriott.com/hotels/maps/travel/ht...,38.422022,-82.445442,"[ This modern, upper-upscale full-service prop...",N/A,4.9,4.6
85,Delta Hotels,Delta Hotels Green Bay,https://www.marriott.com/GRBDE,2750 Ramada Way,Green Bay,Wisconsin,54304,44.476025,-88.077961,N/A,N/A,"[ At Delta Hotels by Marriott Green Bay, we cr...",N/A,4.3,3.9
86,Delta Hotels,Delta Hotels Milwaukee Northwest,https://www.marriott.com/MKEDM,N88 W14750 Main Street,Menomonee Falls,Wisconsin,53051,Reviews,https://www.marriott.com/hotels/maps/travel/mk...,43.177772,-88.093336,[ A seamless experience awaits you at Delta Ho...,N/A,4.0,3.4
87,Delta Hotels,Milwaukee Airport Hotel,https://www.marriott.com/MKEDA,6331 South 13th Street,Milwaukee,Wisconsin,53221,N/A,N/A,N/A,N/A,"[ Welcome to the Milwaukee Airport Hotel, a fu...",N/A,N/A,N/A


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Delta Hotels,Delta Hotels Kunming,https://www.marriott.com/KMGDE,Longhui Road 206,Kunming,,650000,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Kunming is located ...,N/A,N/A,N/A
1,Delta Hotels,Delta Hotels Shanghai Baoshan,https://www.marriott.com/SHABS,"No 1 Lane 888 Haijiang Road, Baoshan District",Shanghai,,200940,Reviews,https://www.marriott.com/hotels/maps/travel/sh...,31.3954603,121.4884866,"[ The hotel started trial opening on Feb 1,201...",N/A,4.7,4.5
2,Delta Hotels,Delta Hotels by Marriott Frankfurt Offenbach,https://www.marriott.com/FRADE,Kaiserleistrasse 45,Offenbach am Main,,63067,Reviews,https://www.marriott.com/hotels/maps/travel/fr...,50.108019,8.73165,[ Make your time in Germany memorable with a s...,N/A,4.7,4.6
3,Delta Hotels,Delta Hotels Istanbul Halic,https://www.marriott.com/ISTDK,Merkez Mahallesi Ayazma Caddesi Cuha Sokak No....,Istanbul,,34406,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Istanbul Halic is l...,N/A,N/A,N/A
4,Delta Hotels,Delta Hotels Cheltenham Chase,https://www.marriott.com/GLODE,Shurdington Road,Brockworth,,GL3 4PB,N/A,N/A,N/A,N/A,[ Delta Hotels by Marriott Cheltenham Chase pr...,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Delta Hotels,Delta Hotels Huntington Downtown,https://www.marriott.com/HTSDH,800 Third Avenue,Huntington,West Virginia,25701,Reviews,https://www.marriott.com/hotels/maps/travel/ht...,38.422022,-82.445442,"[ This modern, upper-upscale full-service prop...",N/A,4.9,4.6
85,Delta Hotels,Delta Hotels Green Bay,https://www.marriott.com/GRBDE,2750 Ramada Way,Green Bay,Wisconsin,54304,44.476025,-88.077961,N/A,N/A,"[ At Delta Hotels by Marriott Green Bay, we cr...",N/A,4.3,3.9
86,Delta Hotels,Delta Hotels Milwaukee Northwest,https://www.marriott.com/MKEDM,N88 W14750 Main Street,Menomonee Falls,Wisconsin,53051,Reviews,https://www.marriott.com/hotels/maps/travel/mk...,43.177772,-88.093336,[ A seamless experience awaits you at Delta Ho...,N/A,4.0,3.4
87,Delta Hotels,Milwaukee Airport Hotel,https://www.marriott.com/MKEDA,6331 South 13th Street,Milwaukee,Wisconsin,53221,N/A,N/A,N/A,N/A,"[ Welcome to the Milwaukee Airport Hotel, a fu...",N/A,N/A,N/A


In [8]:
hotel_df.to_csv('deltahotels_description.csv')